In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.10)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.77 sec - Loss 0.101749 - ACC 80.80% - ACC Mean 80.80% - AUC 88.04% - AUC Mean 88.04% - Deter 000
Ite 00050 - 0.11 sec - Loss 0.095351 - ACC 80.56% - ACC Mean 79.83% - AUC 87.46% - AUC Mean 87.34% - Deter 018
Ite 00100 - 0.11 sec - Loss 0.078948 - ACC 79.77% - ACC Mean 80.50% - AUC 86.62% - AUC Mean 87.36% - Deter 047
Ite 00150 - 0.10 sec - Loss 0.053134 - ACC 79.88% - ACC Mean 80.12% - AUC 87.13% - AUC Mean 87.11% - Deter 097
Ite 00200 - 0.10 sec - Loss 0.038708 - ACC 81.33% - ACC Mean 80.17% - AUC 86.89% - AUC Mean 86.62% - Deter 147
Ite 00250 - 0.11 sec - Loss 0.034882 - ACC 81.20% - ACC Mean 81.18% - AUC 86.92% - AUC Mean 86.76% - Deter 197
Ite 00300 - 0.09 sec - Loss 0.033713 - ACC 80.64% - ACC Mean 80.85% - AUC 86.81% - AUC Mean 86.49% - Deter 247
Ite 00350 - 0.10 sec - Loss 0.033111 - ACC 80.17% - ACC Mean 80.41% - AUC 85.54% - AUC Mean 86.26% - Deter 297
Ite 00400 - 0.09 sec - Loss 0.032666 - ACC 80.47% - ACC Mean 80.60% - AUC 86.29% - AUC Mean 86.54% - Deter 347
I

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 81.34% +- 0.0% - auc: 86.45% +- 0.0%
MEAN - acc: 80.47% +- 0.0% - auc: 87.07% +- 0.0%
KNN - acc: 81.54% +- 0.0% - auc: 88.13% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:35<00:00, 141.79it/s]



GAIN RUN 1/10 - acc: 79.69% +- 1.78% - auc: 87.19% +- 2.16%



100%|██████████| 5000/5000 [00:36<00:00, 136.33it/s]



GAIN RUN 2/10 - acc: 82.41% +- 2.27% - auc: 87.7% +- 3.23%



100%|██████████| 5000/5000 [00:38<00:00, 130.51it/s]


...


GAIN RUN 8/10 - acc: 80.28% +- 2.8% - auc: 88.37% +- 3.66%



100%|██████████| 5000/5000 [00:36<00:00, 136.48it/s]



GAIN RUN 9/10 - acc: 80.98% +- 4.96% - auc: 87.05% +- 2.08%



100%|██████████| 5000/5000 [00:38<00:00, 128.77it/s]



GAIN RUN 10/10 - acc: 81.96% +- 5.27% - auc: 88.33% +- 2.38%

GAIN GLOBAL - acc: 81.04% +- 1.57% - auc: 87.37% +- 0.87%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 78.37% +- 0.0% - auc: 85.48% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 1/10 - acc: 80.0% +- 5.54% - auc: 86.91% +- 4.15%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 2/10 - acc: 81.51% +- 5.97% - auc: 87.53% +- 4.81%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 3/10 - acc: 84.46% +- 2.53% - auc: 90.56% +- 3.32%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 4/10 - acc: 78.88% +- 5.78% - auc: 87.64% +- 4.7%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 5/10 - acc: 82.83% +- 4.49% - auc: 88.55% +- 3.15%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 6/10 - acc: 81.09% +- 3.75% - auc: 87.23% +- 5.71%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 81.33% +- 3.58% - auc: 88.08% +- 2.74%
SOFTIMPUTE RUN 2/10 - acc: 84.9% +- 2.16% - auc: 88.56% +- 3.15%
SOFTIMPUTE RUN 3/10 - acc: 80.64% +- 2.94% - auc: 87.02% +- 2.48%
SOFTIMPUTE RUN 4/10 - acc: 82.1% +- 1.93% - auc: 89.58% +- 2.44%
SOFTIMPUTE RUN 5/10 - acc: 80.01% +- 2.73% - auc: 87.34% +- 3.51%
SOFTIMPUTE RUN 6/10 - acc: 80.37% +- 3.5% - auc: 87.17% +- 2.6%
SOFTIMPUTE RUN 7/10 - acc: 85.95% +- 1.88% - auc: 90.12% +- 3.63%
SOFTIMPUTE RUN 8/10 - acc: 78.42% +- 5.42% - auc: 87.37% +- 3.86%
SOFTIMPUTE RUN 9/10 - acc: 80.64% +- 3.44% - auc: 88.12% +- 2.08%
SOFTIMPUTE RUN 10/10 - acc: 82.84% +- 2.1% - auc: 87.91% +- 2.91%

SOFTIMPUTE GLOBAL - acc: 81.72% +- 2.18% - auc: 88.13% +- 0.98%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.1618

SINKHORN RUN 1/10 - acc: 81.04% +- 5.08% - auc: 85.5% +- 5.43%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.0488

SINKHORN RUN 2/10 - acc: 80.23% +- 3.84% - auc: 87.32% +- 2.25%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.1203

SINKHORN RUN 3/10 - acc: 80.65% +- 5.78% - auc: 85.64% +- 5.23%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.0252

SINKHORN RUN 4/10 - acc: 84.71% +- 4.85% - auc: 88.57% +- 4.41%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.1496

SINKHORN RUN 5/10 - acc: 78.63% +- 5.47% - auc: 85.13% +- 4.5%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.1051

SINKHORN RUN 6/10 - acc: 83.17% +- 2.2% - auc: 87.44% +- 3.47%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.0622

SINKHORN RUN 7/10 - acc: 82.11% +- 3.85% - auc: 88.4% +- 2.07%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.0266

SINKHORN RUN 8/10 - acc: 81.51% +- 1.77% - auc: 87.15

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 80.43% +- 5.25% - auc: 85.81% +- 4.79%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 82.31% +- 3.17% - auc: 88.58% +- 3.16%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 81.17% +- 2.71% - auc: 87.48% +- 3.82%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 79.57% +- 4.39% - auc: 86.38% +- 4.5%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 79.6% +- 3.39% - auc: 85.7% +- 3.8%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 80.3% +- 4.15% - auc: 85.23% +- 3.82%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 80.2% +- 4.06% - auc: 85.32% +- 4.57%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 82.0% +- 4.55% - auc: 88.34% +- 2.67%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 80.

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is even with KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is even with MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is even with SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is even with SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly better than MIDA
